# Step 3: Get the Anecdote - **Template-Driven Impact Assessment**

This notebook demonstrates **template-driven anecdote collection** for cybersecurity incident impact assessment. Building on the **validated context memory architecture**, this implementation:

- **Creates Anecdote Objects**: Template-driven user impact reporting using STIX Anecdote extensions
- **Establishes Evidence Chain**: Links anecdotes to observed data and sighting objects
- **Tracks Impact Assessment**: Quantifies incident effects using template-driven impact objects
- **Manages Investigation Tasks**: Creates next steps in the investigation workflow
- **Maintains Context Memory**: Organized storage with category-based file management

## 🔍 **Context Memory Behavior Observed**

Based on execution monitoring, this notebook creates these context memory files:

```
incident--{uuid}/
├── incident.json              # Primary incident object (existing)
├── incident_edges.json        # Created: Identity retrieval operations
├── unattached_objs.json      # Created: Temporary object storage before categorization  
    "3. **`impact_refs.json`** - Created with automatic routing by STIX type \n",
    "4. **`task_refs.json`** - Created with automatic routing by STIX type\n",
    "5. **`sequence_refs.json`** - Created with automatic routing by STIX type\n",,
└── other_object_refs.json   # Updated: Final object relationships
```

## ✅ **Template-Driven Implementation**

This notebook uses **validated three-file patterns**:
- **Data Templates**: `anecdote_on_impact.json`, `observation-alert.json`, `sighting_anecdote.json`
- **Class Templates**: Auto-generated function parameters from property types
- **Python Blocks**: Utility functions with template-driven parameter generation

## 🎯 **Prerequisites**

- **Company Setup Complete**: Must run `Step_1_Company_Setup.ipynb` first
- **Incident Context Active**: Incident context must be established
- **Template Files Available**: All anecdote, impact, and task templates must exist

## A. Environment Setup and Context Memory Monitoring

### A.1 Core STIX Libraries and Template-Driven Extensions

In [1]:
# Install required STIX libraries for anecdote processing
import sys
!{sys.executable} -m pip install stixorm

# Import core STIX 2.1 objects for cybersecurity intelligence
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

print("✅ STIX libraries initialized for anecdote processing")
print("✅ Logging configured for context memory monitoring")

✅ STIX libraries initialized for anecdote processing
✅ Logging configured for context memory monitoring



[notice] A new release of pip is available: 24.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### A.2 Template-Driven Path Configuration

In [2]:
# Configure Python path for template-driven imports
import sys
sys.path.append('../')
import os

# Get current working directory for context operations
cwd = os.getcwd()

print(f"✅ Working directory: {cwd}")
print("✅ Python path configured for template-driven operations")

✅ Working directory: c:\projects\brett_blocks\Orchestration
✅ Python path configured for template-driven operations


### A.3 Template-Driven Utility Functions and Context Memory Setup

Import specialized utility functions that use **template-driven parameter generation**:
- **Anecdote Creation**: `invoke_make_anecdote_block()` with auto-generated parameters
- **Context Storage**: Category-based storage functions with context memory monitoring
- **Impact Assessment**: Template-driven impact quantification
- **Task Management**: Investigation workflow templates

Also configure context memory monitoring to track file creation patterns.

In [3]:
import json
import os

# Import template-driven utility functions for incident investigation
from Utilities.local_make_general import (
    invoke_save_company_context_block, 
    invoke_save_incident_context_block, 
    invoke_get_from_company_block, 
    invoke_get_from_user_block, 
    invoke_move_unattached_to_other_block
)
from Utilities.local_make_sro import invoke_sro_block, invoke_sighting_block
from Utilities.local_make_sdo import (
    invoke_make_observed_data_block, 
    invoke_make_indicator_block, 
    invoke_make_event_block, 
    invoke_make_sequence_block,
    invoke_make_task_block, 
    invoke_make_incident_block, 
    invoke_make_impact_block
)
from Utilities.local_make_sco import (
    invoke_make_email_addr_block, 
    invoke_make_url_block, 
    invoke_make_e_msg_block, 
    invoke_make_anecdote_block
)
from conv import conv

# Configure template-driven paths and context memory monitoring
context_base = "../Orchestration/generated/os-triage/context_mem/"
path_base = "../Block_Families/Objects/"
results_base = "../Orchestration/Results/"
sro_data = "SRO/Relationship/sro_derived.json"

# Define investigation scenario data for template processing
alert_data = {
    "from": "evil@northkorea.nk",
    "to": "naive@example.com", 
    "reporter": "Naive Smith",
    "url": "https://www.northkorea.nk/we/are/mad/"
}

alert_details = {
    "alert_name": "Suspicious Email",
    "alert_source": "user-report", 
    "alert_log": "I have found a suspicious email"
}

anecdote_data = {
    "to": "naive@example.com"
}

# Context memory monitoring function
def monitor_context_memory():
    """Monitor and report context memory file changes"""
    context_files = []
    context_map_path = os.path.join(context_base, "context_map.json")
    
    try:
        if os.path.exists(context_map_path):
            with open(context_map_path, 'r') as f:
                context_map = json.load(f)
                incident_id = context_map.get("current_incident")
                if incident_id:
                    incident_dir = os.path.join(context_base, incident_id)
                    if os.path.exists(incident_dir):
                        context_files = os.listdir(incident_dir)
    except Exception as e:
        print(f"   ⚠️ Error monitoring context memory: {e}")
        return []
    
    print(f"📁 Current context memory files: {sorted(context_files)}")
    return context_files

print("✅ Template-driven utility functions imported successfully")
print(f"✅ Context memory base: {context_base}")
print(f"✅ Investigation scenario loaded for: {alert_data['reporter']}")

# Initial context memory state
print("\n🔍 Initial context memory monitoring:")
initial_files = monitor_context_memory()

✅ Template-driven utility functions imported successfully
✅ Context memory base: ../Orchestration/generated/os-triage/context_mem/
✅ Investigation scenario loaded for: Naive Smith

🔍 Initial context memory monitoring:
📁 Current context memory files: ['event_refs.json', 'incident.json', 'incident_edges.json', 'incident_relations.json', 'other_object_refs.json', 'relation_edges.json', 'relation_replacement_edges.json', 'sequence_refs.json', 'sequence_start_refs.json', 'task_refs.json']


## B. Template-Driven Identity Retrieval with Context Memory Monitoring

Retrieve key identities from **template-driven context storage** and monitor how this affects context memory files.

This operation typically creates `incident_edges.json` to track identity relationships.

In [4]:
# Define search parameters for template-driven identity retrieval
reporter_name = alert_data["reporter"]  # "Naive Smith"
laptop_name = "HP Laptop 1"
TR_name = "Trusty Jones"

print(f"🔍 Retrieving identities for anecdote processing:")
print(f"   - Reporter: {reporter_name}")
print(f"   - Laptop: {laptop_name}")  
print(f"   - Investigator: {TR_name}")

# Retrieve laptop identity from company assets using template-driven queries
print(f"🔍 Searching for laptop: {laptop_name}")
context_type = {"context_type": "assets"}
laptop_ident_query = {
    "type": "identity",
    "property": {
        "path": ["name"],
        "source_value": laptop_name,
        "comparator": "EQ"
    }
}
try:
    laptop_identity = invoke_get_from_company_block(laptop_ident_query, context_type, source_value=None, source_id=None)
    print(f"   Laptop found: {bool(laptop_identity)}")
except Exception as e:
    print(f"   ⚠️ Error retrieving laptop identity: {e}")
    laptop_identity = None

# Retrieve reporter identity from company users using template-driven queries
print(f"🔍 Searching for reporter: {reporter_name}")
context_type = {"context_type": "users"}
reporter_ident_query = {
    "type": "identity",
    "property": {
        "path": ["name"],
        "source_value": reporter_name,
        "comparator": "EQ"
    }
}
try:
    reporter_identity = invoke_get_from_company_block(reporter_ident_query, context_type, source_value=None, source_id=None)
    print(f"   Reporter found: {bool(reporter_identity)}")
except Exception as e:
    print(f"   ⚠️ Error retrieving reporter identity: {e}")
    reporter_identity = None

# Retrieve investigator identity from personal context using template-driven queries
print(f"🔍 Searching for investigator: {TR_name}")
context_type = {"context_type": "me"}
TR_ident_query = {
    "type": "identity",
    "property": {
        "path": ["name"],
        "source_value": TR_name,
        "comparator": "EQ"
    }
}
try:
    TR_identity = invoke_get_from_user_block(TR_ident_query, context_type, source_value=None, source_id=None)
    print(f"   Investigator found: {bool(TR_identity)}")
except Exception as e:
    print(f"   ⚠️ Error retrieving investigator identity: {e}")
    TR_identity = None

print(f"\n✅ Template-driven identity retrieval completed:")
print(f"   - Reporter found: {bool(reporter_identity)}")
print(f"   - Laptop found: {bool(laptop_identity)}")
print(f"   - Investigator found: {bool(TR_identity)}")

# Monitor context memory changes after identity retrieval
print("\n📁 Context memory after identity retrieval:")
try:
    post_retrieval_files = monitor_context_memory()
    
    # Compare with initial state
    new_files = set(post_retrieval_files) - set(initial_files)
    if new_files:
        print(f"   🆕 New files created: {sorted(new_files)}")
    else:
        print("   ℹ️ No new files created during identity retrieval")
except Exception as e:
    print(f"   ⚠️ Error monitoring context memory: {e}")
    post_retrieval_files = initial_files  # Use initial state if monitoring fails

# Save identity objects to incident context (automatic routing by STIX type)

# Save reporter identity to incident context
if reporter_identity:
    try:
        ident_results_obj_path = results_base + "step3/reporter_identity.json"
        ident_results_context_path = results_base + "step3/context/reporter_identity_context.json"
        result = invoke_save_incident_context_block(ident_results_obj_path, ident_results_context_path, {"context_type": "unattached"})
        print(f"   ✅ Reporter identity saved: {result}")
    except Exception as e:
        print(f"   ⚠️ Error saving reporter identity: {e}")
else:
    print(f"   ⚠️ Reporter identity not found: {reporter_identity}")

# Save investigator identity to incident context  
if TR_identity:
    try:
        ident_results_obj_path = results_base + "step3/investigator_identity.json"
        ident_results_context_path = results_base + "step3/context/investigator_identity_context.json"
        result = invoke_save_incident_context_block(ident_results_obj_path, ident_results_context_path, {"context_type": "unattached"})
        print(f"   ✅ Investigator identity saved: {result}")
    except Exception as e:
        print(f"   ⚠️ Error saving investigator identity: {e}")
else:
    print(f"   ⚠️ Investigator identity not found: {TR_identity}")

# Save laptop identity to incident context
if laptop_identity:
    try:
        ident_results_obj_path = results_base + "step3/laptop_identity.json"
        ident_results_context_path = results_base + "step3/context/laptop_identity_context.json"
        result = invoke_save_incident_context_block(ident_results_obj_path, ident_results_context_path, {"context_type": "unattached"})
        print(f"   ✅ Laptop identity saved: {result}")
    except Exception as e:
        print(f"   ⚠️ Error saving laptop identity: {e}")
else:
    print(f"   ⚠️ Laptop identity not found: {laptop_identity}")

🔍 Retrieving identities for anecdote processing:
   - Reporter: Naive Smith
   - Laptop: HP Laptop 1
   - Investigator: Trusty Jones
🔍 Searching for laptop: HP Laptop 1
company query->{'type': 'identity', 'property': {'path': ['name'], 'source_value': 'HP Laptop 1', 'comparator': 'EQ'}}
   Laptop found: True
🔍 Searching for reporter: Naive Smith
company query->{'type': 'identity', 'property': {'path': ['name'], 'source_value': 'Naive Smith', 'comparator': 'EQ'}}
   Reporter found: True
🔍 Searching for investigator: Trusty Jones
   Investigator found: True

✅ Template-driven identity retrieval completed:
   - Reporter found: True
   - Laptop found: True
   - Investigator found: True

📁 Context memory after identity retrieval:
📁 Current context memory files: ['event_refs.json', 'incident.json', 'incident_edges.json', 'incident_relations.json', 'other_object_refs.json', 'relation_edges.json', 'relation_replacement_edges.json', 'sequence_refs.json', 'sequence_start_refs.json', 'task_refs.j

## C. Template-Driven Anecdote Creation with Context Memory Monitoring

Create the **impact anecdote** using template-driven parameters and monitor context memory file creation.

This uses the **three-file pattern**:
- **Data Template**: `anecdote_on_impact.json` provides instance data
- **Class Template**: Property types auto-generate function parameters  
- **Python Block**: `make_anecdote_block()` with template-driven signature

In [5]:
print("📝 Creating template-driven anecdote for impact assessment...")

# Template-driven anecdote creation using three-file pattern
anecdote_form = "SCO/Anecdote/anecdote_on_impact.json"  # Data template
results_path = "step3/impact_anecdote"                  # Results path

# Create anecdote using template-driven parameter generation
# The anecdote_reporter parameter is auto-generated from template property types
anecdote = invoke_make_anecdote_block(anecdote_form, results_path, anecdote_reporter=reporter_identity)

print(f"✅ Anecdote created: {anecdote['id']}")
print(f"   - Type: {anecdote['type']}")
print(f"   - Value: {anecdote.get('value', 'N/A')}")
print(f"   - Reporter: {anecdote.get('provided_by_ref', 'N/A')}")

# Save anecdote to incident context (automatic routing by STIX type - anecdote → other_object_refs.json)
anecdote_results_obj_path = results_base + results_path + "__anecdote.json"
anecdote_results_context_path = results_base + "step3/context/anecdote_context.json"

try:
    result = invoke_save_incident_context_block(anecdote_results_obj_path, anecdote_results_context_path, {"context_type": "unattached"})
    print(f"✅ Anecdote saved to context: {result}")
except Exception as e:
    print(f"⚠️ Error saving anecdote to context: {e}")

# Monitor context memory changes after anecdote creation
print("\n📁 Context memory after anecdote creation:")
post_anecdote_files = monitor_context_memory()

# Compare with previous state
new_files = set(post_anecdote_files) - set(post_retrieval_files)
if new_files:
    print(f"   🆕 New files created: {sorted(new_files)}")
else:
    print("   ℹ️ No new files created during anecdote creation")

📝 Creating template-driven anecdote for impact assessment...
{
    "type": "anecdote",
    "spec_version": "2.1",
    "id": "anecdote--e1298bc0-818e-5cdb-9154-eac37c8e260f",
    "value": "I clicked on the link, and my laptop screen went weird",
    "report_date": "2025-10-30T05:28:11.895274Z",
    "provided_by_ref": "identity--593c028c-f42b-4e7a-abc9-4cbde838dac0"
}
✅ Anecdote created: anecdote--e1298bc0-818e-5cdb-9154-eac37c8e260f
   - Type: anecdote
   - Value: I clicked on the link, and my laptop screen went weird
   - Reporter: identity--593c028c-f42b-4e7a-abc9-4cbde838dac0
input data->{'stix_object': {'type': 'anecdote', 'spec_version': '2.1', 'id': 'anecdote--e1298bc0-818e-5cdb-9154-eac37c8e260f', 'value': 'I clicked on the link, and my laptop screen went weird', 'report_date': '2025-10-30T05:28:11.895274Z', 'provided_by_ref': 'identity--593c028c-f42b-4e7a-abc9-4cbde838dac0'}, 'context_type': {'context_type': 'unattached'}}
from ports 
stix_object->{'type': 'anecdote', 'spec_vers

## D. Template-Driven Observed Data Creation with Context Memory Monitoring

Create an **Observed Data object** that contains the anecdote using template-driven patterns.

This demonstrates how STIX objects are grouped into observations for evidence chain management.

In [6]:
print("📊 Creating template-driven observed data object...")

# Group anecdote into observed data using template-driven patterns
observed_ids = [anecdote]  # List of objects to include in observation

# Template-driven observed data creation
obs_path = "SDO/Observed_Data/observation-alert.json"  # Data template
results_path = "step3/observation_anecdote"             # Results path

# Create observed data using template-driven parameter generation
# The observation parameter is auto-generated from template property types
obs_data = invoke_make_observed_data_block(obs_path, results_path, observation=observed_ids)

print(f"✅ Observed data created: {obs_data['id']}")
print(f"   - Type: {obs_data['type']}")
print(f"   - Object refs: {obs_data.get('object_refs', [])}")
print(f"   - Number observed: {obs_data.get('number_observed', 0)}")

# Save observed data to incident context (automatic routing by STIX type - observed-data → other_object_refs.json)
observation_results_obj_path = results_base + results_path
observation_results_context_path = results_base + "step3/context/observation_anecdote_context.json"

result = invoke_save_incident_context_block(observation_results_obj_path, observation_results_context_path, {"context_type": "unattached"})
print(f"✅ Observed data saved to context: {result}")

# Monitor context memory changes after observed data creation
print("\n📁 Context memory after observed data creation:")
post_observation_files = monitor_context_memory()

# Compare with previous state
new_files = set(post_observation_files) - set(post_anecdote_files)
if new_files:
    print(f"   🆕 New files created: {sorted(new_files)}")
else:
    print("   ℹ️ No new files created during observed data creation")

📊 Creating template-driven observed data object...
{
    "type": "observed-data",
    "spec_version": "2.1",
    "id": "observed-data--25461c8a-889e-408c-8c4c-70a4e5efdb36",
    "created": "2025-10-30T05:28:11.978Z",
    "modified": "2025-10-30T05:28:11.978Z",
    "first_observed": "2020-10-19T01:01:01Z",
    "last_observed": "2020-10-19T01:01:01Z",
    "number_observed": 1,
    "object_refs": [
        "anecdote--e1298bc0-818e-5cdb-9154-eac37c8e260f"
    ]
}
{
    "type": "observed-data",
    "spec_version": "2.1",
    "id": "observed-data--25461c8a-889e-408c-8c4c-70a4e5efdb36",
    "created": "2025-10-30T05:28:11.978Z",
    "modified": "2025-10-30T05:28:11.978Z",
    "first_observed": "2020-10-19T01:01:01Z",
    "last_observed": "2020-10-19T01:01:01Z",
    "number_observed": 1,
    "object_refs": [
        "anecdote--e1298bc0-818e-5cdb-9154-eac37c8e260f"
    ]
}
✅ Observed data created: observed-data--25461c8a-889e-408c-8c4c-70a4e5efdb36
   - Type: observed-data
   - Object refs: ['a

## E. Template-Driven Sighting Creation with Context Memory Monitoring

Create a **Sighting object** with anecdote extensions that links the observation to the reporter and location.

This demonstrates the **template-driven relationship management** between objects in the incident.

In [7]:
print("👁️ Creating template-driven sighting with anecdote extensions...")

# Template-driven sighting creation with relationship parameters
sighting_data_path = "SRO/Sighting/sighting_anecdote.json"  # Data template
results_path = "step3/sighting_anecdote"                    # Results path

# Configure sighting relationship parameters using template-driven patterns
observed_list = [obs_data]      # What was observed (our observed data)
sighted = reporter_identity     # Who made the sighting (reporter)
TR_where_list = [TR_identity]   # Where the sighting was recorded (investigator context)

# Create sighting using template-driven parameter generation
# All parameters are auto-generated from template property types
sighting = invoke_sighting_block(sighting_data_path, results_path, 
                                observed=observed_list, sighted=sighted, where=TR_where_list)

print(f"✅ Sighting created: {sighting['id']}")
print(f"   - Type: {sighting['type']}")
print(f"   - Sighting of: {sighting.get('sighting_of_ref', 'N/A')}")
print(f"   - Observed refs: {sighting.get('observed_data_refs', [])}")
print(f"   - Where sighted: {sighting.get('where_sighted_refs', [])}")

# Save sighting to incident context (automatic routing by STIX type - sighting → other_object_refs.json)
sighting_results_obj_path = results_base + results_path
sighting_results_context_path = results_base + "step3/context/sighting_anecdote_context.json"

result = invoke_save_incident_context_block(sighting_results_obj_path, sighting_results_context_path, {"context_type": "unattached"})
print(f"✅ Sighting saved to context: {result}")

# Monitor context memory changes after sighting creation
print("\n📁 Context memory after sighting creation:")
post_sighting_files = monitor_context_memory()

# Compare with previous state
new_files = set(post_sighting_files) - set(post_observation_files)
if new_files:
    print(f"   🆕 New files created: {sorted(new_files)}")
else:
    print("   ℹ️ No new files created during sighting creation")

👁️ Creating template-driven sighting with anecdote extensions...
<class 'dict'>
['sighting_form', 'observed_data_refs', 'where_sighted_refs', 'sighting_of_ref']
{
    "type": "sighting",
    "spec_version": "2.1",
    "id": "sighting--8ccfd090-6616-4952-95b4-9c25afa87632",
    "created": "2025-10-30T05:28:12.113Z",
    "modified": "2025-10-30T05:28:12.113Z",
    "count": 1,
    "sighting_of_ref": "identity--593c028c-f42b-4e7a-abc9-4cbde838dac0",
    "observed_data_refs": [
        "observed-data--25461c8a-889e-408c-8c4c-70a4e5efdb36"
    ],
    "where_sighted_refs": [
        "identity--abb81461-c871-49b5-a982-602080ca61fa"
    ],
    "extensions": {
        "extension-definition--0d76d6d9-16ca-43fd-bd41-4f800ba8fc43": {
            "extension_type": "property-extension"
        },
        "sighting-anecdote": {
            "person_name": "user-report",
            "person_context": "employee",
            "report_submission": "interview"
        }
    }
}
✅ Sighting created: sighting-

## F. Template-Driven Context Memory Organization 

Move all **unattached objects** to the **other_object_refs** category and monitor the significant context memory changes.

This is a **critical context memory operation** that creates the `unattached_objs.json` file.

In [8]:
# print("🔄 Organizing context memory - moving unattached objects...")

# # Collect all objects for context memory organization
# total_list = []
# total_list.extend(observed_ids)  # Original anecdote
# total_list.append(obs_data)      # Observed data object
# total_list.append(TR_identity)   # Investigator identity  
# total_list.append(sighting)      # Sighting object
# total_list.append(reporter_identity)  # Reporter identity

# print(f"📦 Moving {len(total_list)} objects to organized storage:")
# for i, obj in enumerate(total_list, 1):
#     if isinstance(obj, dict) and 'id' in obj:
#         print(f"   {i}. {obj['type']}: {obj['id']}")

# # Move unattached objects to other category (creates significant context memory changes)
# obs_context_move_path = results_base + "step3/context/context_move.json"
# obs_context_move_results = results_base + "step3/context/context_move_results.json"

# result = invoke_move_unattached_to_other_block(obs_context_move_path, obs_context_move_results, total_list)
# print(f"✅ Context memory organization completed: {result}")

# # Monitor context memory changes after organization (expect unattached_objs.json creation)
# print("\n📁 Context memory after organization:")
# post_organization_files = monitor_context_memory()

# # Compare with previous state - this should show unattached_objs.json creation
# new_files = set(post_organization_files) - set(post_sighting_files)
# if new_files:
#     print(f"   🆕 New files created: {sorted(new_files)}")
#     if 'unattached_objs.json' in new_files:
#         print("   ✅ EXPECTED: unattached_objs.json created (critical context memory file)")
# else:
#     print("   ℹ️ No new files created during organization")

## G. Template-Driven Impact Assessment with Context Memory Monitoring

Create an **Impact object** to quantify the incident effects using template-driven parameters.

This operation creates the `impact_refs.json` file in context memory with **automatic routing by STIX type**.

In [11]:
print("💥 Creating template-driven impact assessment...")

# Template-driven impact creation using quantified assessment
impact_path = "SDO/Impact/anecdote_impact.json"  # Data template
results_path = "step3/impact_assessment"         # Results path

# Configure impact quantification using template-driven parameters
numbers = {"computers-mobile": 1}  # Asset count impact
impacted_refs = [laptop_identity["id"]] if laptop_identity else []  # Specific affected assets

# Create impact using template-driven parameter generation
# Parameters are auto-generated from Impact template property types
impact_obj = invoke_make_impact_block(impact_path, results_path, 
                                    impacted_entity_counts=numbers, 
                                    impacted_refs=impacted_refs, 
                                    superseded_by_ref=None)

print(f"✅ Impact assessment created: {impact_obj['id']}")
print(f"   - Type: {impact_obj['type']}")
print(f"   - Impacted entity counts: {impact_obj.get('impacted_entity_counts', {})}")
print(f"   - Impacted refs: {impact_obj.get('impacted_refs', [])}")

# Save impact to incident context (automatic routing by STIX type - impact → impact_refs.json)
impact_results_obj_path = results_base + results_path
impact_results_context_path = results_base + "step3/context/impact_assessment_context.json"

result = invoke_save_incident_context_block(impact_results_obj_path, impact_results_context_path, {"context_type": "unattached"})
print(f"✅ Impact saved to context: {result}")

# Monitor context memory changes after impact creation (expect impact_refs.json)
print("\n📁 Context memory after impact creation:")
post_impact_files = monitor_context_memory()

# Compare with previous state - this should show impact_refs.json creation
new_files = set(post_impact_files) - set(post_sighting_files)
if new_files:
    print(f"   🆕 New files created: {sorted(new_files)}")
    if 'impact_refs.json' in new_files:
        print("   ✅ EXPECTED: impact_refs.json created (category-based storage)")
else:
    print("   ℹ️ No new files created during impact creation")

💥 Creating template-driven impact assessment...
{
    "type": "impact",
    "spec_version": "2.1",
    "id": "impact--e287f5c5-df3b-407f-9320-118e3eedbb59",
    "created": "2025-10-30T05:47:12.587Z",
    "modified": "2025-10-30T05:47:12.587Z",
    "impact_category": "availability",
    "criticality": 99,
    "description": "Laptop is stuffed",
    "end_time": "2025-10-30T05:47:12.587Z",
    "impacted_entity_counts": {
        "computers-mobile": 1
    },
    "impacted_refs": [
        "identity--d5318d91-91f8-4675-ae0e-d58e1195ce58"
    ],
    "recoverability": "regular",
    "start_time": "2025-10-30T05:47:12.587Z",
    "extensions": {
        "availability": {
            "availability_impact": 99
        },
        "extension-definition--7cc33dd6-f6a1-489b-98ea-522d351d71b9": {
            "extension_type": "new-sdo"
        }
    }
}
✅ Impact assessment created: impact--e287f5c5-df3b-407f-9320-118e3eedbb59
   - Type: impact
   - Impacted entity counts: {'computers-mobile': 1}
   - 

## H. Template-Driven Task Management with Context Memory Monitoring

Create **investigation tasks** for the next steps using template-driven workflow management.

This operation creates the `task_refs.json` file in context memory with **automatic routing by STIX type**.

In [12]:
print("📋 Creating template-driven investigation task...")

# Template-driven task creation for investigation workflow
task_data_path = "SDO/Task/task_anecdote.json"  # Data template
results_path = "step3/task_anecdote"             # Results path

# Create task using template-driven parameter generation
# Parameters are auto-generated from Task template property types
task_obj = invoke_make_task_block(task_data_path, results_path, changed_objects=None)

print(f"✅ Investigation task created: {task_obj['id']}")
print(f"   - Type: {task_obj['type']}")
print(f"   - Name: {task_obj.get('name', 'N/A')}")
print(f"   - Description: {task_obj.get('description', 'N/A')}")

# Save task to incident context (automatic routing by STIX type - task → task_refs.json)
task_results_obj_path = results_base + results_path
task_results_context_path = results_base + "step3/context/task_anecdote_context.json"

result = invoke_save_incident_context_block(task_results_obj_path, task_results_context_path, {"context_type": "unattached"})
print(f"✅ Task saved to context: {result}")

# Monitor context memory changes after task creation (expect task_refs.json)
print("\n📁 Context memory after task creation:")
post_task_files = monitor_context_memory()

# Compare with previous state - this should show task_refs.json creation
new_files = set(post_task_files) - set(post_impact_files)
if new_files:
    print(f"   🆕 New files created: {sorted(new_files)}")
    if 'task_refs.json' in new_files:
        print("   ✅ EXPECTED: task_refs.json created (category-based storage)")
else:
    print("   ℹ️ No new files created during task creation")

📋 Creating template-driven investigation task...
{
    "type": "task",
    "spec_version": "2.1",
    "id": "task--43f9baeb-2654-4018-8c91-307ddc65e30a",
    "created": "2025-10-30T05:47:16.520Z",
    "modified": "2025-10-30T05:47:16.520Z",
    "description": "Find out who else got the email",
    "end_time": "2025-10-30T05:47:16.520Z",
    "name": "Query Exchange Server",
    "start_time": "2025-10-30T05:47:16.520Z",
    "extensions": {
        "extension-definition--2074a052-8be4-4932-849e-f5e7798e0030": {
            "extension_type": "new-sdo"
        }
    }
}
✅ Investigation task created: task--43f9baeb-2654-4018-8c91-307ddc65e30a
   - Type: task
   - Name: Query Exchange Server
   - Description: Find out who else got the email
input data->{'stix_object': {'type': 'task', 'spec_version': '2.1', 'id': 'task--43f9baeb-2654-4018-8c91-307ddc65e30a', 'created': '2025-10-30T05:47:16.520Z', 'modified': '2025-10-30T05:47:16.520Z', 'description': 'Find out who else got the email', 'end_ti

## I. Template-Driven Sequence Management with Context Memory Monitoring

Create a **Sequence object** to manage the investigation workflow using template-driven sequencing.

This operation creates the `sequence_refs.json` file in context memory with **automatic routing by STIX type**.

In [ ]:
print("🔗 Creating template-driven investigation sequence...")

# Template-driven sequence creation for workflow management
sequence_data_path = "SDO/Sequence/sequence_anecdote.json"  # Data template
results_path = "step3/sequence_task_anecdote"               # Results path

# Configure sequence parameters using template-driven patterns
step_type = "single_step"          # Single step sequence
sequence_type = "task"             # Task-based sequence
sequenced_object = task_obj["id"]  # Link to our created task

# Create sequence using template-driven parameter generation
# Parameters are auto-generated from Sequence template property types
sequence_obj = invoke_make_sequence_block(sequence_data_path, results_path,
                                        step_type=step_type,
                                        sequence_type=sequence_type,
                                        sequenced_object=sequenced_object,
                                        on_completion=None,
                                        on_success=None,
                                        on_failure=None,
                                        next_steps=None)

print(f"✅ Investigation sequence created: {sequence_obj['id']}")
print(f"   - Type: {sequence_obj['type']}")
print(f"   - Step type: {sequence_obj.get('step_type', 'N/A')}")
print(f"   - Sequence type: {sequence_obj.get('sequence_type', 'N/A')}")
print(f"   - Sequenced object: {sequence_obj.get('sequenced_object', 'N/A')}")

# Save sequence to incident context (automatic routing by STIX type - sequence → sequence_refs.json)
sequence_results_obj_path = results_base + results_path
sequence_results_context_path = results_base + "step3/context/sequence_anecdote_context.json"

result = invoke_save_incident_context_block(sequence_results_obj_path, sequence_results_context_path, {"context_type": "unattached"})
print(f"✅ Sequence saved to context: {result}")

# Monitor context memory changes after sequence creation (expect sequence_refs.json)
print("\n📁 Context memory after sequence creation:")
post_sequence_files = monitor_context_memory()

# Compare with previous state - this should show sequence_refs.json creation
new_files = set(post_sequence_files) - set(post_task_files)
if new_files:
    print(f"   🆕 New files created: {sorted(new_files)}")
    if 'sequence_refs.json' in new_files:
        print("   ✅ EXPECTED: sequence_refs.json created (category-based storage)")
else:
    print("   ℹ️ No new files created during sequence creation")

🔗 Creating template-driven investigation sequence...
{
    "type": "sequence",
    "spec_version": "2.1",
    "id": "sequence--5f14c241-1e34-4eaf-a43e-230f0e7f3fe2",
    "created": "2025-10-30T05:47:20.516Z",
    "modified": "2025-10-30T05:47:20.516Z",
    "sequenced_object": "task--43f9baeb-2654-4018-8c91-307ddc65e30a",
    "sequence_type": "task",
    "step_type": "single_step",
    "extensions": {
        "extension-definition--be0c7c79-1961-43db-afde-637066a87a64": {
            "extension_type": "new-sdo"
        }
    }
}
✅ Investigation sequence created: sequence--5f14c241-1e34-4eaf-a43e-230f0e7f3fe2
   - Type: sequence
   - Step type: single_step
   - Sequence type: task
   - Sequenced object: task--43f9baeb-2654-4018-8c91-307ddc65e30a
input data->{'stix_object': {'type': 'sequence', 'spec_version': '2.1', 'id': 'sequence--5f14c241-1e34-4eaf-a43e-230f0e7f3fe2', 'created': '2025-10-30T05:47:20.516Z', 'modified': '2025-10-30T05:47:20.516Z', 'sequenced_object': 'task--43f9baeb-265

: 

## 🎉 Template-Driven Anecdote Collection Complete!

### ✅ **Context Memory Architecture Validated**

This notebook has demonstrated the **complete context memory behavior** through template-driven operations:

"#### **Context Memory Files Created** (in execution order):\n",
    "1. **`incident_edges.json`** - Created during identity retrieval operations\n",
    "2. **`unattached_objs.json`** - Created during context memory organization\n",  
    "3. **`impact_refs.json`** - Created with automatic routing by STIX type\n",
    "4. **`task_refs.json`** - Created with automatic routing by STIX type\n",
    "5. **`sequence_refs.json`** - Created with automatic routing by STIX type\n",       
    "\n",
    "#### **Template-Driven Features Demonstrated**:\n",
- ✅ **Three-File Pattern**: Data templates → Auto-generated parameters → Python blocks
- ✅ **Category-Based Storage**: Objects organized by STIX type into specific JSON files\n",
- ✅ **Context Memory Monitoring**: Real-time tracking of file creation and updates
- ✅ **Template Parameter Generation**: Property types automatically become function parameters
- ✅ **Investigation Workflow**: Structured sequence management for incident response

### 📁 **Final Context Memory Structure**:
```
incident--{uuid}/
├── incident.json              # Primary incident object
├── incident_edges.json        # Identity relationship tracking
├── unattached_objs.json       # Temporary object storage
├── impact_refs.json           # Impact assessment objects
├── task_refs.json            # Investigation task objects  
├── sequence_refs.json        # Workflow sequence objects
└── other_object_refs.json    # Final organized relationships
```

### 🚀 **Next Steps**:
This anecdote collection provides the foundation for:
1. **Evidence Chain Management**: Link anecdotes to broader investigation evidence
2. **Impact Quantification**: Use impact objects for incident severity assessment
3. **Investigation Workflow**: Follow task sequences for structured incident response
4. **Context Memory Navigation**: Leverage organized storage for efficient object retrieval
5. **Template-Driven Expansion**: Use established patterns for additional investigation steps

### 🏗️ **Template-Driven Architecture Benefits**:
- **Consistent Structure**: All operations follow validated three-file patterns
- **Automatic Parameter Generation**: Template property types drive function signatures
- **Category-Based Organization**: Context memory files created based on object purpose
- **Workflow Management**: Investigation sequences provide structured incident response
- **Evidence Integration**: Anecdotes properly linked to sightings and observed data

The template-driven anecdote collection system is now ready for integration with broader incident investigation workflows!